In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "lower_grade_glioma_and_glioblastoma"
cohort = "GSE35158"

# Input paths
in_trait_dir = "../../input/GEO/lower_grade_glioma_and_glioblastoma"
in_cohort_dir = "../../input/GEO/lower_grade_glioma_and_glioblastoma/GSE35158"

# Output paths
out_data_file = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/GSE35158.csv"
out_gene_data_file = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/gene_data/GSE35158.csv"
out_clinical_data_file = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/clinical_data/GSE35158.csv"
json_path = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE35158_family.soft.gz', 'GSE35158_series_matrix.txt.gz']
Identified SOFT files: ['GSE35158_family.soft.gz']
Identified matrix files: ['GSE35158_series_matrix.txt.gz']

Background Information:
!Series_title	"Expression profiling of lower-grade diffuse astrocytic glioma"
!Series_summary	"Diffuse gliomas represent the most prevalent class of primary brain tumor.  Despite significant recent advances in the understanding of glioblastoma (WHO IV), its most malignant subtype, lower-grade (WHO II and III) glioma variants remain comparatively understudied, especially in light of their notably variable clinical behavior.  To examine the foundations of this heterogeneity, we performed multidimensional molecular profiling, including global transcriptional analysis, on 101 lower-grade diffuse astrocytic gliomas collected at our own institution, and validated our findings using publically available gene expression and copy number data from large independent patient

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the background information, this dataset mentions "global transcriptional analysis"
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Trait Data Availability
# Based on the sample characteristics, we can use 'who grade' at index 1 to distinguish between
# lower-grade glioma (grade II) and glioblastoma (grade III)
trait_row = 1  # 'who grade: III', 'who grade: II'

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert WHO grade to binary trait: 1 for glioblastoma (Grade III), 0 for lower-grade glioma (Grade II)"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert WHO grade to binary
    if value == 'III':
        return 1  # glioblastoma
    elif value == 'II':
        return 0  # lower-grade glioma
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion, not used in this dataset"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion, not used in this dataset"""
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create DataFrame from the sample characteristics dictionary provided in previous step output
    # The sample characteristics dictionary format is:
    # {0: ['tumor type: normal brain', 'tumor type: diffuse astrocytic glioma'], 
    #  1: [nan, 'who grade: III', 'who grade: II'], ...}
    
    # First, let's load the GEO series matrix file to get the sample IDs and characteristics
    sample_ids = []
    sample_chars = []
    
    with gzip.open(f"{in_cohort_dir}/GSE35158_series_matrix.txt.gz", 'rt') as file:
        for line in file:
            if line.startswith("!Sample_geo_accession"):
                sample_ids = line.strip().split('\t')[1:]
            elif line.startswith("!Sample_characteristics_ch1"):
                chars = line.strip().split('\t')[1:]
                sample_chars.append(chars)
    
    # Create a DataFrame with samples as rows and characteristic types as columns
    # Transpose the data so samples are rows and characteristic types are columns
    clinical_df = pd.DataFrame(index=sample_ids)
    
    # From the previous output, we know the trait info is at row 1 in sample_characteristics dictionary
    if len(sample_chars) > trait_row:
        clinical_df[trait_row] = sample_chars[trait_row]
    
    # Extract clinical features using geo_select_clinical_features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of processed clinical data:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")


Preview of processed clinical data:
{1: [nan]}
Saved clinical data to ../../output/preprocess/lower_grade_glioma_and_glioblastoma/clinical_data/GSE35158.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651235',
       'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651254',
       'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278', 'ILMN_1651285',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651315', 'ILMN_1651316',
       'ILMN_1651336', 'ILMN_1651343', 'ILMN_1651346', 'ILMN_1651347'],
      dtype='object', name='ID')

Gene expression data shape: (20792, 81)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers, these are ILMN_ prefixed IDs which are Illumina microarray probe IDs
# These are not human gene symbols and will need to be mapped to gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
              GSM864095  GSM864096  GSM864097  GSM864098  GSM864099
ID                                                                 
ILMN_1651209    7.08126    7.02804    7.04731    6.89578    7.54783
ILMN_1651228   12.26640   11.81630   12.25260   12.76770   12.55510
ILMN_1651229   11.51870   11.18470   10.52200   10.83890   10.72500
ILMN_1651235   11.06620   11.11270   12.09500   12.04060   11.62470
ILMN_1651236    6.71231    6.80484    7.71360    6.76902    6.83577



Platform information:
!Series_title = Expression profiling of lower-grade diffuse astrocytic glioma
!Platform_title = Illumina HumanHT-12 WG-DASL V4.0 R2 expression beadchip
#Definition = Gene description from the source
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_description = FFPE material/DASL processed
!Sample_descr


Gene annotation columns:
['ID', 'Transcript', 'Species', 'Source', 'Search_Key', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']

Gene annotation preview:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-0

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns to use for mapping
# From previous steps, we observed that:
# - The gene expression data uses 'ILMN_*' identifiers which are Illumina probe IDs
# - The gene annotation data has an 'ID' column with these same identifiers
# - The 'Symbol' column appears to contain gene symbols

prob_col = 'ID'        # Probe identifier column
gene_col = 'Symbol'    # Gene symbol column

# 2. Create gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Display the first few rows of the mapping
print("Preview of gene mapping:")
print(gene_mapping.head())
print(f"Total mappings: {len(gene_mapping)}")

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols (standardize capitalization, handle synonyms)
gene_data = normalize_gene_symbols_in_index(gene_data)

# Display the results
print("\nAfter mapping and normalization:")
print(f"Number of genes: {gene_data.shape[0]}")
print(f"Number of samples: {gene_data.shape[1]}")
print("First 5 gene symbols:")
print(gene_data.index[:5])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Saved gene expression data to {out_gene_data_file}")


Preview of gene mapping:
             ID        Gene
0  ILMN_3166687  ERCC-00162
1  ILMN_3165566  ERCC-00071
2  ILMN_3164811  ERCC-00009
3  ILMN_3165363  ERCC-00053
4  ILMN_3166511  ERCC-00144
Total mappings: 29377



After mapping and normalization:
Number of genes: 19449
Number of samples: 81
First 5 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Saved gene expression data to ../../output/preprocess/lower_grade_glioma_and_glioblastoma/gene_data/GSE35158.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (19449, 81)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/lower_grade_glioma_and_glioblastoma/gene_data/GSE35158.csv
Loaded clinical data:
                                      1
lower_grade_glioma_and_glioblastoma NaN
Transposed clinical data to correct format:
   lower_grade_glioma_and_glioblastoma
1                                  NaN
Number of common samples between clinical and genetic data: 0
Clinical data index: ['1']
Gene data columns: ['GSM864095', 'GSM864096', 'GSM864097', 'GSM864098', 'GSM864099', '...']
Extracted 81 GSM IDs from gene data.
Created new clinical data with matching sample IDs:
           lower_grade_glioma_and_glioblastoma
GSM864095                                    1
GSM864096                                    1
GSM864097                                    1
GSM864098                                    1
GSM864099                                    1
Gene data shape for linking (samples as rows): (81, 19449)
Linked data shape: (81, 19450)
Linked data preview (

Linked data shape after handling missing values: (81, 19450)
For the feature 'lower_grade_glioma_and_glioblastoma', the least common label is '1' with 14 occurrences. This represents 17.28% of the dataset.
The distribution of the feature 'lower_grade_glioma_and_glioblastoma' in this dataset is fine.

Is trait biased: False
Data quality check result: Usable


Linked data saved to ../../output/preprocess/lower_grade_glioma_and_glioblastoma/GSE35158.csv
